### Week 4 HW

1. VGG19 

In [10]:
#vgg는 네트워크의 깊이를 깊게 만드는 것이 성능에 어떤 영향을 미치는지 확인하고자 함. 
#그래서 컨볼루션 필터의 사이즈가  가장 작은 3*3
#필터사이즈가 크면  클수록 이미지의 사이즈가 빨리 축소되어가지고 네트워크의 깊이 충분히 깊게 만들기 불가능.
# vgg16 구조 : 16 weight layer 근데 우리가 하고자 하는건 vgg 19. 
#input conv3-64 두 개 
#maxpool conv3-128 두 개 
#maxpool conv3-256 네 개 
#maxpool conv3-512 네 개 
#maxpool conv3-512 네 개 

#근데 cfg 16 + 3개로 바꾸면..? 그니까

In [12]:
!pip install tensorflow

     |████████████████████████████████| 165.2 MB 95 kB/s  eta 0:00:01   |                                | 501 kB 759 kB/s eta 0:03:37     |▏                               | 1.0 MB 759 kB/s eta 0:03:37     |████▊                           | 24.3 MB 148 kB/s eta 0:15:46     |███████▍                        | 38.2 MB 310 kB/s eta 0:06:50     |█████████                       | 46.9 MB 93 kB/s eta 0:20:59     |██████████▋                     | 54.9 MB 157 kB/s eta 0:11:40     |███████████▏                    | 57.8 MB 571 kB/s eta 0:03:08     |████████████▍                   | 63.8 MB 511 kB/s eta 0:03:19     |█████████████▌                  | 69.7 MB 339 kB/s eta 0:04:41     |██████████████▏                 | 73.1 MB 237 kB/s eta 0:06:28     |██████████████▉                 | 76.5 MB 381 kB/s eta 0:03:53     |████████████████▍               | 84.6 MB 121 kB/s eta 0:11:05     |███████████████████████▌        | 121.4 MB 1.2 MB/s eta 0:00:38     |██████████████████████████      | 133.8 MB 69

In [16]:
import tensorflow
from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions

vgg = VGG19()
vgg.summary()

574717952/574710816 [==============================] - 107s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

In [3]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import visdom

import warnings
warnings.filterwarnings('ignore')

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [29]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train = True, download=True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform =transform)
testloader =  torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=0)

classes = ('plane','car', 'bird','cat',
           'deer', 'dog','frog', 'horse', 'ship', 'truck')

#Num_classes = 10

Files already downloaded and verified
Files already downloaded and verified


In [30]:
import torchvision.models.vgg as vgg

In [31]:
cfg = [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M']


In [32]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512*1*1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 9)

In [33]:
vgg19 = VGG(vgg.make_layers(cfg), 10, True).to(device)

In [34]:
a = torch.Tensor(1,3,32,32).to(device)
out = vgg19(a)
print(out)

tensor([[ 6.7528e+29,  1.1084e+30, -4.3379e+30,  1.8694e+30,  3.7388e+30,
          3.0958e+30, -1.3266e+30,  1.4053e+30,  2.7549e+29,  1.1439e+30]],
       grad_fn=<AddmmBackward>)


In [35]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg19.parameters(), lr=0.005, momentum=0.9)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [37]:
print(len(trainloader))
epochs = 2 #epoch 2로 바꾸기
for epoch in range(epochs):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get tue inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg19(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 ==29:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss/30))
            running_loss = 0.0

print('Finished Training')

98
[1,    30] loss: 2.303
[1,    60] loss: 2.303
[1,    90] loss: 2.303
[2,    30] loss: 2.303
[2,    60] loss: 2.303
[2,    90] loss: 2.303
Finished Training


2. ResNet 34 

In [51]:
#block 은 basic block 사용하고, layer은 [3,4,6,3]사용
# Layer 깊어질수록 더 성능이 좋아지는가 실험. 그런데 그 고점이 존재한다는 사실을 깨달음. 
#망이 깊어지면 gradient vanishing, exploding, degradation 발생. 
#복잡도 줄이기 위해 max-pooling, hidden fc, dropout 등 사용 안함.

In [39]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)

print(trainset.data.shape)

train_data_mean = trainset.data.mean( axis=(0,1,2) )
train_data_std = trainset.data.std( axis=(0,1,2) )


print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [40]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer','dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [41]:
import torchvision.models.resnet as resnet

In [42]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [43]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace = True)

        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                for m in self.modules():
                    if isinstance(m, Bottleneck):
                        nn.init.constant_(m.bn3.weight, 0)
                    elif isinstance(m, BasicBlock):
                        nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [44]:
resnet34 = ResNet(resnet.BasicBlock, [3,4,6,3], 10, True).to(device) 

#Basicblock이랑 [3,4,6,3] 반영, num_classes = 10.

In [48]:
a = torch.Tensor(1,3,32,32).to(device)
out = resnet34(a)
print(out)

tensor([[-0.3311, -0.5377,  0.6676,  0.5659,  0.7112, -0.3386, -0.3186,  0.0189,
         -0.5236, -0.3373]], grad_fn=<AddmmBackward>)


In [49]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet34.parameters(), lr = 0.1, momentum = 0.9, weight_decay = 5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [50]:
print(len(trainloader))
epochs = 2

for epoch in range(epochs):

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the unputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0

print('Finished Training')

196
[1,    30] loss: 1.120
[1,    60] loss: 1.107
[1,    90] loss: 1.099
[1,   120] loss: 1.015
[1,   150] loss: 1.006
[1,   180] loss: 1.032
[2,    30] loss: 0.930
[2,    60] loss: 0.907
[2,    90] loss: 0.916
[2,   120] loss: 0.872
[2,   150] loss: 0.856
[2,   180] loss: 0.884
Finished Training
